<a href="https://colab.research.google.com/github/fgsantosti/programacaodispositivosmoveis/blob/main/App_Flutter_15_CriandoAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para criar uma **API simples em Dart** e um **aplicativo Flutter** que consuma essa API localmente, você pode usar as seguintes etapas. Vamos organizar isso em duas partes:

---

## **1. Criando a API Local com Dart**

A API será feita usando o pacote **`shelf`**, que é simples e eficiente para criar servidores HTTP no Dart. Esta API retornará dados simulados de uma lista de álbuns.

### **Configuração da API**
1. **Crie um novo projeto Dart** usando o comando:
   ```bash
   dart create api_local
   cd api_local
   ```

2. **Adicione o pacote `shelf` e `shelf_router`** no arquivo `pubspec.yaml`:
   ```yaml
   dependencies:
     shelf: ^1.4.1
     shelf_router: ^1.1.4
   ```

   Depois rode:
   ```bash
   dart pub get
   ```

3. **Crie o arquivo `bin/server.dart`** com o seguinte conteúdo:

```dart
import 'dart:io';
import 'package:shelf/shelf.dart';
import 'package:shelf/shelf_io.dart' as io;
import 'package:shelf_router/shelf_router.dart';

void main() async {
  // Criação de um router para definir endpoints
  final router = Router();

  // Endpoint: GET /albums - Retorna uma lista de álbuns simulada
  router.get('/albums', (Request request) {
    final albums = [
      {'userId': 1, 'id': 1, 'title': 'Album 1'},
      {'userId': 1, 'id': 2, 'title': 'Album 2'},
      {'userId': 2, 'id': 3, 'title': 'Album 3'},
    ];
    return Response.ok(
      jsonEncode(albums),
      headers: {'Content-Type': 'application/json'},
    );
  });

  // Endpoint: GET /albums/<id> - Retorna um álbum específico
  router.get('/albums/<id>', (Request request, String id) {
    final album = {'userId': 1, 'id': int.parse(id), 'title': 'Album $id'};
    return Response.ok(
      jsonEncode(album),
      headers: {'Content-Type': 'application/json'},
    );
  });

  // Adicionando o router ao handler
  final handler = Pipeline().addMiddleware(logRequests()).addHandler(router);

  // Inicia o servidor local na porta 8080
  final server = await io.serve(handler, InternetAddress.loopbackIPv4, 8080);
  print('Servidor rodando em http://${server.address.host}:${server.port}');
}
```

4. **Execute o servidor**:
   ```bash
   dart run bin/server.dart
   ```
   Você verá no terminal a mensagem indicando que o servidor está rodando em **`http://127.0.0.1:8080`**.

---

## **2. Criando o Aplicativo Flutter para Consumir a API Local**

Agora vamos criar o aplicativo Flutter que faz requisições HTTP à API Dart criada acima.

### **Configuração do Projeto Flutter**
1. **Crie um novo projeto Flutter**:
   ```bash
   flutter create flutter_api_local
   cd flutter_api_local
   ```

2. **Adicione a dependência `http`** no `pubspec.yaml`:
   ```yaml
   dependencies:
     http: ^1.1.0
   ```

   Rode o comando:
   ```bash
   flutter pub get
   ```

3. **Edite o arquivo `lib/main.dart`** com o seguinte conteúdo:

```dart
import 'dart:convert';
import 'package:flutter/material.dart';
import 'package:http/http.dart' as http;

void main() {
  runApp(const MyApp());
}

class MyApp extends StatefulWidget {
  const MyApp({super.key});

  @override
  State<MyApp> createState() => _MyAppState();
}

class _MyAppState extends State<MyApp> {
  List<dynamic> albums = [];

  // Função para buscar os álbuns da API
  Future<void> fetchAlbums() async {
    final url = Uri.parse('http://127.0.0.1:8080/albums'); // API local
    try {
      final response = await http.get(url);
      if (response.statusCode == 200) {
        setState(() {
          albums = json.decode(response.body);
        });
      } else {
        print('Erro ao carregar os álbuns: ${response.statusCode}');
      }
    } catch (e) {
      print('Erro: $e');
    }
  }

  @override
  void initState() {
    super.initState();
    fetchAlbums();
  }

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'API Local Example',
      home: Scaffold(
        appBar: AppBar(
          title: const Text('Consumo de API Local'),
        ),
        body: albums.isEmpty
            ? const Center(child: CircularProgressIndicator())
            : ListView.builder(
                itemCount: albums.length,
                itemBuilder: (context, index) {
                  final album = albums[index];
                  return ListTile(
                    title: Text(album['title']),
                    subtitle: Text('ID: ${album['id']} | User ID: ${album['userId']}'),
                  );
                },
              ),
        floatingActionButton: FloatingActionButton(
          onPressed: fetchAlbums,
          child: const Icon(Icons.refresh),
        ),
      ),
    );
  }
}
```

---

## **Execução Simultânea da API e Flutter**

1. **Inicie a API Dart**:
   No terminal onde está o projeto Dart:
   ```bash
   dart run bin/server.dart
   ```

2. **Execute o Aplicativo Flutter**:
   No terminal onde está o projeto Flutter:
   ```bash
   flutter run -d chrome
   ```

---

## **Comportamento Esperado**

- A API Dart estará rodando em **`http://127.0.0.1:8080`**.
- O aplicativo Flutter fará uma requisição para a API local e exibirá a lista de álbuns retornada.
- A lista será exibida com título, ID e User ID.

---

### **Observação Importante**
- Se estiver rodando no emulador Android, você deve substituir **`127.0.0.1`** por **`10.0.2.2`**, que é o IP do host para o emulador:
   ```dart
   final url = Uri.parse('http://10.0.2.2:8080/albums');
   ```

### **Permitir Acesso Externo na API Dart**
Por padrão, a API Dart só permite conexões locais. Para permitir acesso por outros dispositivos ou emuladores, altere a inicialização do servidor no bin/server.dart:

```dart
final server = await io.serve(handler, InternetAddress.anyIPv4, 8080);
```
Isso faz o servidor escutar em todos os endereços IPv4 da máquina.
